In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium 
import requests
import json
from geopy.distance import geodesic

hdb_resale_prices = pd.read_csv("final_hdb_resale_prices.csv")

<h3>Data Preparation</h3>

In [ ]:
hdb_resale_prices.drop(columns=['index'], inplace=True)
hdb_resale_prices.head()

<h5><b>Checking for Missing Values</b></h5>

There are no missing values in the dataset.

In [ ]:
hdb_resale_prices.isnull().sum()

<h5><b>Reason for Null Values for Resale Application and Demand Ratio</b></h5>

As shown below, the 2150 number of null values for 'resale application' and 'demand ratio' matches the number of resale flats in 2023. There is no data available for the demand for resale flats in 2023. Hence, the resale application and demand ratio is null for 2023.

In [ ]:
hdb_resale_prices.groupby('year').count()

In [ ]:
hdb_resale_prices.info()

In [ ]:
hdb_resale_prices.describe()

<h1>Exploratory Data Analysis</h1>

### Distribution of Resale Prices
<h5><b>Use mean or median resale price?</b></h5>

As seen in the plotted histogram for the resale prices from 2017 to 2023, the resale price data is right skewed. Since the price data does not follow a symmetric distribution, we will use the <i><b>median price</b></i> instead of the mean price to represent the average resale price of the flats.

The right-skewed KDE plot also suggests that the mean prices are greater than the median.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.boxplot(x=hdb_resale_prices['resale_price'])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.histplot(data=hdb_resale_prices, x="resale_price", ax=ax, kde=True)

<h3>Flat Type Distribution</h3>

Based on the plotted bar chart, the majority of the resale flats are 4-room flats and multi-generation flats account for the least number of resale flats.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.countplot(x=hdb_resale_prices['flat_type'], data=hdb_resale_prices)
abs_values = hdb_resale_prices['flat_type'].value_counts(ascending=False).values
ax.bar_label(container=ax.containers[0], labels=abs_values)
plt.show()

Since the young are more likely to purchase 3,4,5 room and executive flats we decided to drop the records of 1 room, 2room and multi-gen flats.

In [ ]:
# drop 1room, 2room, and multi-generation flats
hdb_resale_prices = hdb_resale_prices[hdb_resale_prices['flat_type'] != '1 ROOM']
hdb_resale_prices = hdb_resale_prices[hdb_resale_prices['flat_type'] != '2 ROOM']
hdb_resale_prices = hdb_resale_prices[hdb_resale_prices['flat_type'] != 'MULTI-GENERATION']

hdb_resale_prices.reset_index(inplace=True)

In [ ]:
hdb_resale_prices.head()

In [ ]:
# plot distribution of prices against flat type 
fig, ax = plt.subplots(figsize=(10, 10))
sns.boxplot(x=hdb_resale_prices['flat_type'], y=hdb_resale_prices['resale_price'])

### Investigating the distribution of flat_model

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.countplot(y='flat_model', data=hdb_resale_prices)
abs_values = hdb_resale_prices['flat_model'].value_counts(ascending=False).values
ax.bar_label(container=ax.containers[0], labels=abs_values)
plt.show()

# Feature Engineering

Based on the studies and analysis performed by other researchers, the following features that have been concluded to be <b>important</b> in determining the prices of HDB resale flats are as follows: 
- Time Taken to Travel to Central Business District (CBD), Raffles Place
- Number of Years Left on Lease
- Maturity of Estate
<br>

We are also considering <b>other factors</b> that have not been explored by other reseearchers. The factors that we have identified are as follows: 
- Singapore's Inflation Rate
- Flat Model
- Floor Level
- Prices of BTO
- Number of BTO Flats Launched that Year
- Demand Ratio in Resale Flat


Before adding these features into our dataset, we will first perform some data cleaning and feature engineering on the existing features in our dataset.

- convert the flat condition with quantitative measures
- combine the resale prices and the sqm to find the $/sqm (still need to keep sqm to quantify the size!!) 
- remaining lease should be converted to a ratio 
- combine the block and street name to obtain the lat,long - from there we can obtain the distance from amentities, distance from public transport, distance from central
- using the town --- can also do some quantitative measures to quantify how mature the estate is
- average price of the houses of the same category in the area from here we derive the estimated COV
- demand-scarcity ratio --- not sure how to quantify this
- total sale price of the hdb
- there should be a way to convert the storey range to scoring value


<b>DISCLAIMER: </b> Some of the feature engineering code can be found in resale_data_processing.ipynb. They have been shifted there for ease of reading and efficiency purposes.

In [ ]:
#add year column to dataframe
hdb_resale_prices['year'] = pd.DatetimeIndex(hdb_resale_prices['month']).year

### Calculating Price Per Square Meter

In [ ]:
# add new column to store adjusted resale price 
hdb_resale_prices['cpi_adjusted_price_per_sqm'] = hdb_resale_prices["cpi_adjusted_price"] / hdb_resale_prices["floor_area_sqm"]
hdb_resale_prices.head()

### Demand for resale


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.countplot(x=hdb_resale_prices['year'], data=hdb_resale_prices)
abs_values = hdb_resale_prices['year'].value_counts(ascending=False).values
ax.bar_label(container=ax.containers[0], labels=abs_values)
ax.set_title('Number of Resale Applications Per Year')
plt.show()

<h3>IsMature Estate</h3>

In [ ]:
# list_mature_estates = ["ang mo kio", 
#                         "bishan", 
#                         "clementi",
#                         "bukit merah",
#                         "bedok",
#                         "bukit timah",
#                         "geylang",
#                         "central",
#                         "marine parade",
#                         "queenstown",
#                         "pasir ris",
#                         "kallang" , "whampoa", "marine parade" , "serangoon", "tampines", "toa payoh"]

# for i in range(len(hdb_resale_prices)):
#     if hdb_resale_prices.loc[i,'town'].lower() in list_mature_estates:
#         hdb_resale_prices.loc[i,'isMatureEstate'] = 1
#     else:
#         hdb_resale_prices.loc[i,'isMatureEstate'] = 0

In [ ]:
# hdb_resale_prices.to_csv('final_hdb_resale_prices.csv')

### Correlation

In [ ]:
plt.figure(figsize=(16, 6))
corr_df = hdb_resale_prices.corr()
hm = sns.heatmap(corr_df, annot=True)

### Investigate if lease is an important factor

In [ ]:
import csv
hdb_resale_listings = []

with open('final_hdb_resale_prices.csv', newline='', encoding="utf8") as f:
    reader = csv.reader(f)
    for row in reader:
        hdb_resale_listings.append(row)

    # Remove header
    hdb_resale_listings_headers = hdb_resale_listings[0]
    hdb_resale_listings = hdb_resale_listings[1:]
    
print(hdb_resale_listings[0]) # Print first row of data

In [ ]:
average_resale_for_lease_year_remaining = {}
for data in hdb_resale_listings:
    if round(float(data[16])/12) not in average_resale_for_lease_year_remaining:
        average_resale_for_lease_year_remaining[round(float(data[16])/12)] = []
        average_resale_for_lease_year_remaining[round(float(data[16])/12)].append(float(data[7]))
    else:
        average_resale_for_lease_year_remaining[round(float(data[16])/12)].append(float(data[7]))
    

In [ ]:
for item in average_resale_for_lease_year_remaining:
    avg = round(sum(average_resale_for_lease_year_remaining[item])/len(average_resale_for_lease_year_remaining[item]))
    average_resale_for_lease_year_remaining[item] = avg

In [ ]:
lease = list(average_resale_for_lease_year_remaining.keys())
average_resale_value = list(average_resale_for_lease_year_remaining.values())

## Write your code below
fig,ax = plt.subplots(figsize=(18,12))
ax.plot(lease,average_resale_value,marker='d')
ax.set_ylabel('Resale Value')
ax.set_xlabel('Lease Remaining')
ax.set_title('Average Resale Value for Lease Years Remaining')
plt.show()


### Investigate how floor area correlate with price

In [ ]:
floor_area_sqm = []
resale_price = []
for data in hdb_resale_listings:
    floor_area_sqm.append(float(data[5]))
    resale_price.append(float(data[7]))
fig,ax = plt.subplots(figsize=(18,12))
ax.scatter(floor_area_sqm,resale_price)
ax.set_xlabel('Floor Area (in square metres)')
ax.set_ylabel('Sale Price (in million SGD)')
ax.set_title('Relationship between Floor Area and Resale Price')
plt.show()


### Investigating resale counts by town

In [ ]:
resale_counts_by_town = {}  # Populate this dictionary with counts

import operator
## Write your code below

for data in hdb_resale_listings:
    hdb_town = data[2]
    
    if hdb_town in resale_counts_by_town:
        
        resale_counts_by_town[hdb_town] += 1 
    
    else:
        
        resale_counts_by_town[hdb_town] = 1

    
        
sorted_resale_counts_by_town = dict(sorted(resale_counts_by_town.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
resale_counts_sorted_desc = sorted_resale_counts_by_town
neighbourhood_listings = list(resale_counts_sorted_desc.keys())
num_listings = list(resale_counts_sorted_desc.values())
# Import matplotlib library here

import matplotlib.pyplot as plt
fig,ax = plt.subplots(figsize=(18,12))
ax.bar(neighbourhood_listings,num_listings)
ax.set_title('Counts of Resale Flats from Jan 2017 to June 2020')
ax.set_xticklabels(neighbourhood_listings,rotation = 90)
ax.set_ylabel('Number of Resale Flats')
ax.set_xlabel('Town')
plt.show()

# Folium Clusters

<h3>3 Room</h3>

In [ ]:
import folium
from folium.plugins import HeatMap
lat_long = []
## Write your code below
for data in hdb_resale_listings:
    if data[3] == '3 ROOM':
        lat_long.append([data[8],data[9]])
map_folium = folium.Map(location=[1.357,103.826], height=350, width=800,zoom_start=11.4)

HeatMap(lat_long, radius=8, gradient={0.2:'blue', 0.4:'purple', 0.6:'orange', 1.0:'red'}).add_to(map_folium) 

display(map_folium)

In [ ]:
from folium.plugins import MarkerCluster   
## Write your code below (Note: the Folium map output might take a while to load)
map_folium = folium.Map(location=[1.357,103.826], height=550, width=900, zoom_start=11.5)

marker_cluster = MarkerCluster().add_to(map_folium)
for data in hdb_resale_listings:
    if data[3] == '3 ROOM':
        lat_long_listing = [data[8],data[9]]
        pop_display_price = '$' + str(data[6])
        tooltipmsg = 'Click to view price'

        folium.Marker(lat_long_listing,popup=pop_display_price,tooltip = tooltipmsg).add_to(marker_cluster)

display(map_folium)


<h3>4 Room</h3>

In [ ]:
import folium
from folium.plugins import HeatMap
lat_long = []
## Write your code below
for data in hdb_resale_listings:
    if data[3] == '4 ROOM':
        lat_long.append([data[8],data[9]])
map_folium = folium.Map(location=[1.357,103.826], height=350, width=800,zoom_start=11.4)

HeatMap(lat_long, radius=8, gradient={0.2:'blue', 0.4:'purple', 0.6:'orange', 1.0:'red'}).add_to(map_folium) 

display(map_folium)



In [ ]:
from folium.plugins import MarkerCluster   
## Write your code below (Note: the Folium map output might take a while to load)
map_folium = folium.Map(location=[1.357,103.826], height=550, width=900, zoom_start=11.5)

marker_cluster = MarkerCluster().add_to(map_folium)
for data in hdb_resale_listings:
    if data[3] == '4 ROOM':
        lat_long_listing = [data[8],data[9]]
        pop_display_price = '$' + str(data[6])
        tooltipmsg = 'Click to view price'

        folium.Marker(lat_long_listing,popup=pop_display_price,tooltip = tooltipmsg).add_to(marker_cluster)

display(map_folium)


<h3>5 Room</h3>

In [ ]:
import folium
from folium.plugins import HeatMap
lat_long = []
## Write your code below
for data in hdb_resale_listings:
    if data[3] == '5 ROOM':
        lat_long.append([data[8],data[9]])
map_folium = folium.Map(location=[1.357,103.826], height=350, width=800,zoom_start=11.4)

HeatMap(lat_long, radius=8, gradient={0.2:'blue', 0.4:'purple', 0.6:'orange', 1.0:'red'}).add_to(map_folium) 

display(map_folium)

In [ ]:
from folium.plugins import MarkerCluster   
## Write your code below (Note: the Folium map output might take a while to load)
map_folium = folium.Map(location=[1.357,103.826], height=550, width=900, zoom_start=11.5)

marker_cluster = MarkerCluster().add_to(map_folium)
for data in hdb_resale_listings:
    if data[3] == '5 ROOM':
        lat_long_listing = [data[8],data[9]]
        pop_display_price = '$' + str(data[6])
        tooltipmsg = 'Click to view price'

        folium.Marker(lat_long_listing,popup=pop_display_price,tooltip = tooltipmsg).add_to(marker_cluster)

display(map_folium)

<h3>Executive</h3>

In [ ]:
import folium
from folium.plugins import HeatMap
lat_long = []
## Write your code below
for data in hdb_resale_listings:
    if data[3] == 'EXECUTIVE':
        lat_long.append([data[8],data[9]])
map_folium = folium.Map(location=[1.357,103.826], height=350, width=800,zoom_start=11.4)

HeatMap(lat_long, radius=8, gradient={0.2:'blue', 0.4:'purple', 0.6:'orange', 1.0:'red'}).add_to(map_folium) 

display(map_folium)

In [ ]:
from folium.plugins import MarkerCluster   
## Write your code below (Note: the Folium map output might take a while to load)
map_folium = folium.Map(location=[1.357,103.826], height=550, width=900, zoom_start=11.5)

marker_cluster = MarkerCluster().add_to(map_folium)
for data in hdb_resale_listings:
    if data[3] == 'EXECUTIVE':
        lat_long_listing = [data[8],data[9]]
        pop_display_price = '$' + str(data[6])
        tooltipmsg = 'Click to view price'

        folium.Marker(lat_long_listing,popup=pop_display_price,tooltip = tooltipmsg).add_to(marker_cluster)

display(map_folium)

<h1>Preliminary Models</h1>


In [ ]:
#split training and test data